# Paper figures - in vitro multiome data

In [2]:
here::i_am("rna_atac/trajectory/01_RNA_WT_trajectory_inference.ipynb")

source(here::here("settings.R"))
source(here::here("utils.R"))

suppressPackageStartupMessages(library(scran))
suppressPackageStartupMessages(library(scater))
suppressPackageStartupMessages(library(dplyr))

BPPARAM <- BiocParallel::bpparam()
BPPARAM$workers = 21

# Multi core using future - built in to seurat
# plan("multicore", workers = 16)
options(future.globals.maxSize = 50 * 1024 ^ 3) # for 50 Gb RAM

here() starts at /rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/code



In [3]:
args = list()

# Metadata
args$metadata = file.path(io$basedir, 'results/rna_atac/clustering/metadata_celltype_annotated.txt.gz')

# RNA_sce
args$rna_sce = file.path(io$basedir, 'processed/rna/SingleCellExperiment.rds')

# outdir
args$outdir = file.path(io$basedir, 'results/rna_atac/figures/')
dir.create(args$outdir, recursive=TRUE, showWarnings =FALSE)

In [3]:
args = list()
# ATAC
args$atac_metadata = file.path(io$basedir, 'results/atac/archR/qc/sample_metadata_update_PeakClusters.txt.gz')
args$atac_lsi = file.path(io$basedir, 'results/atac/archR/dimensionality_reduction/cells/PeakMatrix/remove_ExE_cells_FALSE/batch_correction_None/lsi_nfeatures25000_dims20.txt.gz')
# RNA
args$rna_metadata = file.path(io$basedir, '/results/rna/mapping/sample_metadata_after_mapping.txt.gz')
args$rna_pca = file.path(io$basedir, 'results/rna/dimensionality_reduction/sce/none/pca_features1500_pcs15.txt.gz')
args$rna_sce = file.path(io$basedir, 'processed/rna/SingleCellExperiment.rds')

# outdir
args$outdir = file.path(io$basedir, 'results/rna_atac/dimensionality_reduction/')
dir.create(args$outdir, recursive=TRUE, showWarnings =FALSE)

In [4]:
##########################
## Load sample metadata ##
##########################

metadata_atac <- fread(args$atac_metadata) %>%
  .[pass_atacQC==TRUE & doublet_call==FALSE | pass_atacQC==TRUE & is.na(doublet_call)] %>% 
  .[,exp := str_replace_all(sample, opts$sample2exp)]

metadata_rna <- fread(args$rna_metadata) %>%
  .[pass_rnaQC==TRUE & doublet_call==FALSE] %>% 
  .[,exp := str_replace_all(sample, opts$sample2exp)]

In [5]:
nrow(metadata_atac)

[1] 35864

In [6]:
nrow(metadata_rna)

[1] 35125